In [40]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import tree
from pyearth import Earth
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

# import matplotlib.pyplot as plt

In [26]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [27]:
def load_both_data(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file/' + project + '.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr'],axis = 1)
    else:
        X = X
    return X,y


def load_data_commit_level(project,metric):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file/' + project + '.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    commit_hash = df.commit_hash
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
       'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
       'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
       'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr'],axis = 1)
    else:
        X = X
    
    df = X
    df['Bugs'] = y
    df['commit_hash'] = commit_hash
    unique_commits = df.commit_hash.unique()
    
    train_df = df[df.commit_hash.isin(unique_commits[0:int(len(unique_commits)*.6)])]
    test_df = df[df.commit_hash.isin(unique_commits[int(len(unique_commits)*.6):])]
    
    train_df.reset_index(inplace=True, drop=True)
    test_df.reset_index(inplace=True, drop=True)
    train_y = train_df.Bugs
    train_X = train_df.drop(['Bugs','commit_hash'],axis = 1)
    
    return train_X,train_y,test_df


In [28]:
def run_self_release(project,metric):
    precision = []
    recall = []
    pf = []
    f1 = []
    g_score = []
    auc = []
    pci_20 = []
    ifa = []
    X_train,y_train,test_df = load_data_commit_level(project,metric)      
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    unique_commits_list = np.array_split(test_df.commit_hash.unique(), 5)
    for i in range(len(unique_commits_list)):
        test_df_subset = test_df[test_df.commit_hash.isin(unique_commits_list[i])]
        y_test = test_df_subset.Bugs
        X_test = test_df_subset.drop(['Bugs','commit_hash'],axis = 1)
        if metric == 'process':
            loc = X_test['file_la'] + X_test['file_lt']
        elif metric == 'product':
            loc = X_test.CountLineCode
        else:
            loc = X_test['file_la'] + X_test['file_lt']                
        predicted = clf.predict(X_test)
        abcd = metrices.measures(y_test,predicted,loc)
        pf.append(abcd.get_pf())
        recall.append(abcd.calculate_recall())
        precision.append(abcd.calculate_precision())
        f1.append(abcd.calculate_f1_score())
        g_score.append(abcd.get_g_score())
        pci_20.append(abcd.get_pci_20())
        ifa.append(abcd.get_ifa())
        try:
            auc.append(roc_auc_score(y_test, predicted))
        except:
            auc.append(0)
        print(classification_report(y_test, predicted))
#         print(recall,precision,pf,f1,g_score,auc,pci_20)
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [41]:
def run_self_k(project,metric):
    precision = []
    recall = []
    pf = []
    f1 = []
    g_score = []
    auc = []
    pci_20 = []
    ifa = []
    importance = []
    X,y = load_both_data(project,metric)
    for _ in range(5):
        skf = StratifiedKFold(n_splits=5)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            if metric == 'process':
                loc = X_test['file_la'] + X_test['file_lt']
            elif metric == 'product':
                loc = X_test.CountLineCode
            else:
                loc = X_test['file_la'] + X_test['file_lt']
            df_smote = pd.concat([X_train,y_train],axis = 1)
            df_smote = apply_smote(df_smote)
            y_train = df_smote.Bugs
            X_train = df_smote.drop('Bugs',axis = 1)
            clf = tree.DecisionTreeClassifier()
            clf.fit(X_train,y_train)
            importance = 0 #clf.feature_importances_ # change for RF
            predicted = clf.predict(X_test)
            abcd = metrices.measures(y_test,predicted,loc)
            pf.append(abcd.get_pf())
            recall.append(abcd.calculate_recall())
            precision.append(abcd.calculate_precision())
            f1.append(abcd.calculate_f1_score())
            g_score.append(abcd.get_g_score())
            pci_20.append(abcd.get_pci_20())
            ifa.append(abcd.get_ifa())
            try:
                auc.append(roc_auc_score(y_test, predicted))
            except:
                auc.append(0)
            print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [50]:
def run_self(project,metric):
    X,y = load_both_data(project,metric)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    if metric == 'process':
        loc = X_test['file_la'] + X_test['file_lt']
    elif metric == 'product':
        loc = X_test.CountLineCode
    else:
        loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = tree.DecisionTreeClassifier()
    clf.fit(X_train,y_train)
    importance = 0 #clf.feature_importances_ # change for RF
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [51]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [52]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects[150:]:
    try:
        if project == '.DS_Store':
            continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project,'process')
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
final_result['featue_importance'] = featue_importance

+++++++++++++++++   spring-xd-samples  +++++++++++++++++
Shape of passed values is (62, 45), indices imply (62, 66)
+++++++++++++++++   environment-dashboard  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         9
           1       1.00      0.10      0.18        10

    accuracy                           0.53        19
   macro avg       0.75      0.55      0.42        19
weighted avg       0.76      0.53      0.41        19

+++++++++++++++++   plantuml  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1936
           1       0.98      0.98      0.98       987

    accuracy                           0.99      2923
   macro avg       0.98      0.98      0.98      2923
weighted avg       0.99      0.99      0.99      2923

+++++++++++++++++   egads  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00         4

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18

+++++++++++++++++   libsignal-service-java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       194
           1       0.95      0.90      0.92        20

    accuracy                           0.99       214
   macro avg       0.97      0.95      0.96       214
weighted avg       0.99      0.99      0.99       214

+++++++++++++++++   MarkdownView  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        38
           1       0.92      0.92      0.92        12

    accuracy                           0.96        50
   macro avg  

              precision    recall  f1-score   support

           0       0.92      0.96      0.94        23
           1       0.93      0.88      0.90        16

    accuracy                           0.92        39
   macro avg       0.93      0.92      0.92        39
weighted avg       0.92      0.92      0.92        39

+++++++++++++++++   HwTxtReader  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00        29

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        68
weighted avg       1.00      1.00      1.00        68

+++++++++++++++++   Recyclerview-Gallery  +++++++++++++++++
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        29
           1       0.87      1.00      0.93        13

    accuracy                           0.95        42
   macro avg     

              precision    recall  f1-score   support

           0       0.81      0.81      0.81        16
           1       0.70      0.70      0.70        10

    accuracy                           0.77        26
   macro avg       0.76      0.76      0.76        26
weighted avg       0.77      0.77      0.77        26

+++++++++++++++++   Launcher  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       149
           1       0.92      0.96      0.94        50

    accuracy                           0.97       199
   macro avg       0.95      0.97      0.96       199
weighted avg       0.97      0.97      0.97       199

+++++++++++++++++   gs-uploading-files  +++++++++++++++++
Shape of passed values is (32, 45), indices imply (32, 66)
+++++++++++++++++   android-flowlayout  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.62      0.56      0.59         9
           1  

              precision    recall  f1-score   support

           0       1.00      0.33      0.50         6
           1       0.82      1.00      0.90        18

    accuracy                           0.83        24
   macro avg       0.91      0.67      0.70        24
weighted avg       0.86      0.83      0.80        24

+++++++++++++++++   SmoothProgressBar  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        25
           1       0.88      0.93      0.90        15

    accuracy                           0.93        40
   macro avg       0.92      0.93      0.92        40
weighted avg       0.93      0.93      0.93        40

+++++++++++++++++   JustAuth  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        58
           1       1.00      0.92      0.96        38

    accuracy                           0.97        96
   macro avg       0.98

...

              precision    recall  f1-score   support

           0       0.98      0.93      0.96       497
           1       0.83      0.95      0.88       165

    accuracy                           0.94       662
   macro avg       0.90      0.94      0.92       662
weighted avg       0.94      0.94      0.94       662

+++++++++++++++++   ice-demos  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.75      0.74      0.75       213
           1       0.79      0.79      0.79       258

    accuracy                           0.77       471
   macro avg       0.77      0.77      0.77       471
weighted avg       0.77      0.77      0.77       471

+++++++++++++++++   librec  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       481
           1       0.95      0.92      0.93       512

    accuracy                           0.93       993
   macro avg       0.93      0.93

Shape of passed values is (1064, 45), indices imply (1064, 66)
+++++++++++++++++   handlebars.java  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       566
           1       0.94      0.94      0.94       378

    accuracy                           0.95       944
   macro avg       0.95      0.95      0.95       944
weighted avg       0.95      0.95      0.95       944

+++++++++++++++++   stringtemplate4  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       436
           1       0.87      0.88      0.87       320

    accuracy                           0.89       756
   macro avg       0.89      0.89      0.89       756
weighted avg       0.89      0.89      0.89       756

+++++++++++++++++   RxNetty  +++++++++++++++++
Shape of passed values is (4407, 45), indices imply (4407, 66)
+++++++++++++++++   ghprb-plugin  +++++++++++++++++
           

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1897
           1       0.96      0.97      0.96       999

    accuracy                           0.97      2896
   macro avg       0.97      0.97      0.97      2896
weighted avg       0.97      0.97      0.97      2896

+++++++++++++++++   TrebleShot  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1310
           1       0.98      0.97      0.98       907

    accuracy                           0.98      2217
   macro avg       0.98      0.98      0.98      2217
weighted avg       0.98      0.98      0.98      2217

+++++++++++++++++   interview  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       305
           1       0.90      0.79      0.84        77

    accuracy                           0.94       382
   macro avg       0.92      

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       750
           1       0.94      0.93      0.93       467

    accuracy                           0.95      1217
   macro avg       0.95      0.95      0.95      1217
weighted avg       0.95      0.95      0.95      1217

+++++++++++++++++   download-manager  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       682
           1       0.86      0.92      0.89       331

    accuracy                           0.92      1013
   macro avg       0.91      0.92      0.92      1013
weighted avg       0.93      0.92      0.93      1013

+++++++++++++++++   AppiumTestDistribution  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       272
           1       0.91      0.88      0.89       330

    accuracy                           0.89       602
   macro a

Shape of passed values is (5962, 45), indices imply (5962, 66)
+++++++++++++++++   thingsboard  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       936
           1       0.99      0.99      0.99      1296

    accuracy                           0.98      2232
   macro avg       0.98      0.98      0.98      2232
weighted avg       0.98      0.98      0.98      2232

+++++++++++++++++   blueflood  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       889
           1       0.94      0.95      0.95       607

    accuracy                           0.96      1496
   macro avg       0.96      0.96      0.96      1496
weighted avg       0.96      0.96      0.96      1496

+++++++++++++++++   androidannotations  +++++++++++++++++
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1332
           1    

In [ ]:
# with open('results/Performance/commit_guru_file_specific/process+product_700_rf_release.pkl', 'wb') as handle:
#     pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)